In [1]:
import os
import datetime
import pandas as pd
import numpy as np
from robi.example_spring import get_spring_data
from src.sindy_utils import library_size
from src.training import train_network
import tensorflow as tf

# Generate data

In [2]:
training_data = get_spring_data(20)
validation_data = get_spring_data(3)

(1000, 2)
(1000, 2)
(1000, 2)
(1000, 2)
(1000, 2)
(1000, 2)
(1000, 2)
(1000, 2)
(1000, 2)
(1000, 2)
(1000, 2)
(1000, 2)
(1000, 2)
(1000, 2)
(1000, 2)
(1000, 2)
(1000, 2)
(1000, 2)
(1000, 2)
(1000, 2)
(1000, 2)
(1000, 2)
(1000, 2)


# Set up model and training parameters

In [3]:
params = {}

params['input_dim'] = training_data['x'].shape[-1]
params['latent_dim'] = 1
params['model_order'] = 2
params['poly_order'] = 3
params['include_sine'] = True
params['library_dim'] = library_size(2*params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500
params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_x'] = 5e-4
params['loss_weight_sindy_z'] = 5e-5
params['loss_weight_sindy_regularization'] = 1e-5

params['activation'] = 'sigmoid'
params['widths'] = [128,64,32]

# training parameters
params['epoch_size'] = training_data['x'].shape[0]
params['batch_size'] = 1000
params['learning_rate'] = 1e-4

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 100

# training time cutoffs
params['max_epochs'] = 5001
params['refinement_epochs'] = 1001

# Run training experiments

In [ ]:
num_experiments = 10
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))

    params['save_name'] = 'spring_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")

    tf.compat.v1.reset_default_graph()

    results_dict = train_network(training_data, validation_data, params)
    df = df.append({**results_dict, **params}, ignore_index=True)

df.to_pickle('experiment_results_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')

EXPERIMENT 0
TRAINING
Epoch 0
   training loss 0.03469647467136383, (0.032734137, 2.513918, 3.6533194, 0.99821264)
   validation loss 0.0330548956990242, (0.03220249, 2.591093, 1.4257354, 0.99821264)
decoder loss ratio: 3.949013, decoder SINDy loss  ratio: 1.000006
Epoch 100
   training loss 0.005685549229383469, (0.003850192, 5.1382245e-05, 3.6533105, 0.869966)
   validation loss 0.002875443547964096, (0.002153879, 2.3454082e-05, 1.4257274, 0.869966)
decoder loss ratio: 0.264132, decoder SINDy loss  ratio: 1.000000
Epoch 200
   training loss 0.005685468204319477, (0.003851601, 4.405945e-05, 3.6533105, 0.72099334)
   validation loss 0.0028743401635438204, (0.0021542655, 2.2589345e-05, 1.4257274, 0.72099334)
decoder loss ratio: 0.264179, decoder SINDy loss  ratio: 1.000000
Epoch 300
   training loss 0.005684082396328449, (0.0038519148, 6.174328e-05, 3.6533105, 0.55093414)
   validation loss 0.0028727443423122168, (0.0021543698, 3.2544045e-05, 1.4257274, 0.55093414)
decoder loss ratio: 0